In [1]:
import os

from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")
credential = DefaultAzureCredential()

# Initialize Azure AI project and Azure OpenAI conncetion with your environment variables
azure_ai_project = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("AZURE_RESOURCE_GROUP_NAME"),
    "project_name": os.environ.get("AZURE_PROJECT_NAME"),
}
print(azure_ai_project)

model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
}

print(model_config)
from azure.ai.evaluation import GroundednessProEvaluator, GroundednessEvaluator

# Initialzing Groundedness and Groundedness Pro evaluators
groundedness_eval = GroundednessEvaluator(model_config)
groundedness_pro_eval = GroundednessProEvaluator(azure_ai_project=azure_ai_project, credential=credential)

query_response = dict(
    query="Which tent is the most waterproof?",
    context="The Alpine Explorer Tent is the most water-proof of all tents available.",
    response="The Alpine Explorer Tent is the most waterproof."
)

# Running Groundedness Evaluator on a query and response pair
groundedness_score = groundedness_eval(
    **query_response
)
print(groundedness_score)

groundedness_pro_score = groundedness_pro_eval(
    **query_response
)
print(groundedness_pro_score)

{'subscription_id': 'e0493f49-bc5c-4207-a643-9b5f6503a36d', 'resource_group_name': 'rg-llmops-dev', 'project_name': 'ai-project-nhbcfrc3qjxb6'}
{'azure_endpoint': 'https://aoai-nhbcfrc3qjxb6.openai.azure.com', 'api_key': '6d876834436c4ca7bf8478b93bff390e', 'azure_deployment': 'gpt-4o', 'api_version': '2024-10-01-preview'}


Class GroundednessProEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


{'groundedness': 5.0, 'gpt_groundedness': 5.0, 'groundedness_reason': 'The response is fully correct and complete, directly addressing the query with precise information from the context.'}
{'groundedness_pro_label': True, 'groundedness_pro_reason': 'All Contents are grounded'}


In [2]:
# Conversation mode
import json

conversation_str =  """{"messages": [ 
    { "content": "Which tent is the most waterproof?", "role": "user" }, 
    { "content": "The Alpine Explorer Tent is the most waterproof", "role": "assistant", "context": "From the our product list the alpine explorer tent is the most waterproof. The Adventure Dining Table has higher weight." }, 
    { "content": "How much does it cost?", "role": "user" }, { "content": "$120.", "role": "assistant", "context": "The Alpine Explorer Tent is $120."} 
] }""" 
conversation = json.loads(conversation_str)

groundedness_conv_score = groundedness_eval(conversation=conversation)
print(groundedness_conv_score)

{'groundedness': 5.0, 'gpt_groundedness': 5.0, 'evaluation_per_turn': {'groundedness': [5.0, 5.0], 'gpt_groundedness': [5.0, 5.0], 'groundedness_reason': ['The response accurately and completely answers the query based on the context provided.', 'The response is fully correct and complete, directly addressing the query with precise information from the context.']}}


In [3]:
from azure.ai.evaluation import ViolenceEvaluator
credential = DefaultAzureCredential()


# Initializing Violence Evaluator with project information
violence_eval = ViolenceEvaluator(credential=credential, azure_ai_project=azure_ai_project)
# Running Violence Evaluator on a query and response pair
violence_score = violence_eval(query="What is the capital of France?", response="Paris.")
# print(json.dump(violence_score,indent=4))
print(violence_score)

# Conversation mode


conversation_str =  """{"messages": [ { "content": "Which tent is the most waterproof?", "role": "user" }, { "content": "The Alpine Explorer Tent is the most waterproof", "role": "assistant", "context": "From the our product list the alpine explorer tent is the most waterproof. The Adventure Dining Table has higher weight." }, { "content": "How much does it cost?", "role": "user" }, { "content": "$120.", "role": "assistant", "context": "The Alpine Explorer Tent is $120."} ] }""" 
conversation = json.loads(conversation_str)

violence_conv_score = violence_eval(conversation=conversation) 

print(json.dumps(violence_conv_score, indent=4))

Class ViolenceEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


{'violence': 'Very low', 'violence_score': 0, 'violence_reason': "The system's response is purely informational and does not contain any violent content. It simply provides the capital of France, which is Paris. There is no mention of violence, harm, or any related topics."}
{
    "violence_score": 0.0,
    "evaluation_per_turn": {
        "violence": [
            "Very low",
            "Very low"
        ],
        "violence_score": [
            0,
            0
        ],
        "violence_reason": [
            "The system's response is a straightforward answer to a question about waterproof tents. There is no mention of violence, harm, or any content that could be considered distressing or discomforting. The response is purely informational and does not contain any violent content.",
            "The system's response does not contain any violent content. It simply provides a price in response to the human's question. There is no mention or depiction of violence, physical harm, 

In [4]:
conversation_str =  """{"messages": [ { "content": "Which tent is the most waterproof?", "role": "user" }, { "content": "The Alpine Explorer Tent is the most waterproof", "role": "assistant", "context": "From the our product list the alpine explorer tent is the most waterproof. The Adventure Dining Table has higher weight." }, { "content": "How much does it cost?", "role": "user" }, { "content": "$120.", "role": "assistant", "context": "The Alpine Explorer Tent is $120."} ] }""" 
conversation = json.loads(conversation_str)

groundedness_conv_score = groundedness_eval(conversation=conversation)
print(groundedness_conv_score)

{'groundedness': 5.0, 'gpt_groundedness': 5.0, 'evaluation_per_turn': {'groundedness': [5.0, 5.0], 'gpt_groundedness': [5.0, 5.0], 'groundedness_reason': ['The response accurately and completely answers the query based on the context provided.', 'The response is fully correct and complete, directly addressing the query with precise information from the context.']}}


In [5]:
class AnswerLengthEvaluator:
    def __init__(self):
        pass

    def __call__(self, *, answer: str, **kwargs):
        return {"answer_length": len(answer)}
    
answer_length = AnswerLengthEvaluator()(answer="What is the speed of light?")

print(answer_length)    

{'answer_length': 27}


In [6]:
import os
import json
import sys
from promptflow.client import load_flow

class FriendlinessEvaluator:
    def __init__(self, model_config):
       
        prompty_path = os.path.join(os.getcwd(), "friendliness.prompty")
        self._flow = load_flow(source=prompty_path, model={"configuration": model_config})

    def __call__(self, *, response: str, **kwargs):
        llm_response = self._flow(response=response)
        try:
            response = json.loads(llm_response)
        except Exception as ex:
            response = llm_response
        return response

In [7]:

friendliness_eval = FriendlinessEvaluator(model_config)

friendliness_score = friendliness_eval(response="I will not apologize for my behavior!")
print(friendliness_score)

{'score': 1, 'reason': 'The response is defensive and lacks warmth or friendliness.'}


In [ ]:
from azure.ai.evaluation import evaluate

answer_length_eval = AnswerLengthEvaluator()

result = evaluate(
    data="data.jsonl", # provide your data here
    evaluators={
        "groundedness": groundedness_eval,
        # "answer_length": answer_length_eval
    },
    # column mapping
    evaluator_config={
        "groundedness": {
            "column_mapping": {
                "query": "${data.question}",
                "context": "${data.context}",
                "response": "${data.answer}"
            } 
        }
    },
    # Optionally provide your Azure AI project information to track your evaluation results in your Azure AI project
    azure_ai_project=azure_ai_project,
    # Optionally provide an output path to dump a json of metric summary, row level data and metric and Azure AI project URL
    output_path="./myevalresults.json"
)

print(result)

[2024-12-18 06:51:12 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2024-12-18 06:51:12 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_dul380s_20241218_065112_634257, log path: /home/andy/.promptflow/.runs/azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_dul380s_20241218_065112_634257/logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_dul380s_20241218_065112_634257


In [13]:
%pip install azure-identity azure-ai-projects azure-ai-ml

In [14]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import Evaluation, Dataset, EvaluatorConfiguration, ConnectionType
from azure.ai.evaluation import F1ScoreEvaluator, RelevanceEvaluator, ViolenceEvaluator

# Load your Azure OpenAI config
deployment_name = os.environ.get("AZURE_OPENAI_DEPLOYMENT")
api_version = os.environ.get("AZURE_OPENAI_API_VERSION")
conn_str = os.environ.get("AZURE_AI_PROJECT_CONN_STR")
# Create an Azure AI Client from a connection string. Avaiable on Azure AI project Overview page.
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=conn_str
)

In [15]:
data_id, _ = project_client.upload_file("./myevalresults.json")